In [34]:
import simpy
import random
import numpy as np
from simpy.events import Event

In [35]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

SHIFT_DURATION=480

In [36]:
answered_calls = 0

In [37]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
        
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [38]:
def Customer(name, env):
    global answered_calls
    
    if robocall.count >= robocall.capacity:
        return # Call dropped
    
    with robocall.request() as req:
        yield req

        print("%s's call connected at %g" % (name, env.now))
        answered_calls += 1
        # Voice recognition and routing
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            operator = operator1
        else:
            operator = operator2

    # Voice recognition failure
    if random.uniform(0,1) < .1:
        print('%s hangs up the call at %g' % (name, env.now))
        return 

    print('%s is rooted to operator %d at %g' % (name,(operator==operator2)+1, env.now))

    with operator.request(0) as req:

        result = yield req | env.timeout(10)
        # Reneging for 10 mins
        if req not in result:
            print('%s is left the operator %d after 10 minutes at %g' % (name,
                                                           (operator==operator2)+1,
                                                           env.now))
            return 
        print('%s is assigned to the operator %d at %g' % (name,
                                                           (operator==operator2)+1,
                                                           env.now))
        yield env.process(service(env,operator))
        print('%s is done with operator %d at %g' % (name,
                                                    (operator==operator2)+1,
                                                    env.now))

In [39]:
#Break request times and sorting
def gen_break_times(env):
    break_count = np.random.poisson(8)
    opr_break_times = []
    for _ in range(break_count):
        opr_break_times.append(random.uniform(env.now,env.now+SHIFT_DURATION))
    opr_break_times.sort()
    return opr_break_times

In [40]:
def Operator(env, opr, number, shift_number):
    opr_break_times = gen_break_times(env)
    shift_end = (shift_number+1) * SHIFT_DURATION
    #opr_2_break_times.insert(0,44)
    #opr_2_break_times.insert(1,46) These are for test.
    #print(opr_2_break_times)
    #opr_1_break_times.insert(opr_1_break, SHIFT_DURATION-1)
    #opr_1_break_times.insert(opr_1_break, SHIFT_DURATION+5) For test
    #print(opr_1_break_times)
    for break_time in opr_break_times:
        if env.now > break_time:
            yield env.timeout(env.now - break_time)
        with opr.request(1) as brk:
            yield brk | env.timeout(shift_end - env.now)

            if shift_end > env.now: # Got a break
                print("break started for operator %d at %g" % (number , env.now))
                yield env.timeout(min(3, shift_end-env.now))
                print("break ended for operator %d at %g" % (number , env.now))
            else: # End of shift
                return

In [41]:
def customer_generator(env):
    i = 1
    while answered_calls < 1000:
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        yield env.process(Customer('Customer %d' % i, env))
        
        i += 1
    finished.success()

In [42]:
def shift_generator(env):
    shift_number = 0
    
    while True:
        env.process(Operator(env, operator1, 1, shift_number))
        env.process(Operator(env, operator2, 2, shift_number))
        
        yield env.timeout(8*60)
        shift_number += 1

In [43]:
env = simpy.Environment()
operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)
robocall  = simpy.Resource(env, capacity = 100)
finished  = Event(env)
env.process(customer_generator(env))
env.process(shift_generator(env))
env.run(finished)
print(env.now)

break started for operator 1 at 0
break started for operator 2 at 0
break ended for operator 1 at 3
break ended for operator 2 at 3
break started for operator 1 at 3
break started for operator 2 at 3
break ended for operator 1 at 6
break ended for operator 2 at 6
break started for operator 1 at 6
break started for operator 2 at 6
break ended for operator 1 at 9
break ended for operator 2 at 9
break started for operator 1 at 9
break started for operator 2 at 9
break ended for operator 1 at 12
break ended for operator 2 at 12
break started for operator 1 at 12
break started for operator 2 at 12
break ended for operator 1 at 15
break ended for operator 2 at 15
break started for operator 1 at 15
break started for operator 2 at 15
break ended for operator 1 at 18
break ended for operator 2 at 18
break started for operator 1 at 18
break started for operator 2 at 18
break ended for operator 1 at 21
break ended for operator 2 at 21
break started for operator 1 at 21
break started for operator 

AttributeError: 'Event' object has no attribute 'success'